In [19]:
! pip install torchsummary

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from matplotlib import pyplot as plt
from time import time
from torch.utils.data import DataLoader
from torchsummary import summary

torch.__version__

You should consider upgrading via the 'pip install --upgrade pip' command.


'1.7.1'

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
batch_size = 64

train_dataset = datasets.MNIST(root='data/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='data/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(test_dataset, batch_size, shuffle=True)

8.1%

100.1%

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


0.5%5%

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


180.4%

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw
Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


In [38]:
def accuracy(model, data_loader):
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for batch_idx, (x, y) in enumerate(data_loader):
            x, y = x.to(device), y.to(device)
            y_pred = model(x).argmax(1)
            num_correct += (y == y_pred).sum()
            num_samples += y.size(0)
    acc = num_correct / num_samples
    return acc

def train(model, data_loader, num_epochs, learning_rate):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fun = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        model.train()
        start_time = time()
        for batch_idx, (x, y) in enumerate(data_loader):
            x, y = x.to(device), y.to(device)
            scores = model(x)
            loss = loss_fun(scores, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'epoch {epoch}, {time() - start_time:.1f}s: {accuracy(model, data_loader):.1%}')

In [44]:
class NN(nn.Module):
    def __init__(self, input_size=784, num_classes=10):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.loss = nn.CrossEntropyLoss()
        # self.half().to(device)
        self.to(device)

    def forward(self, x): 
        # x = x.half().to(device)
        x = x.to(device)
        x = x.reshape(x.shape[0], -1) 
        x = F.relu(self.fc1(x)) 
        x = self.fc2(x) 
        return x

num_epochs = 2 
learning_rate = 1e-3
    
model = NN()
summary(model, (1, 28, 28))
train(model, train_loader, num_epochs, learning_rate)
print(f'test: {accuracy(model, test_loader):.1%}')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]          50,240
            Linear-2                   [-1, 10]             650
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.19
Estimated Total Size (MB): 0.20
----------------------------------------------------------------
epoch 0, 8.5s: 93.6%
epoch 1, 8.7s: 95.6%
test: 95.3%


In [43]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.relu = nn.ReLU()
        self.pool = nn.AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5, 5), stride=(1, 1), padding=(0, 0))
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=(5, 5), stride=(1, 1), padding=(0, 0))
        self.linear1 = nn.Linear(120, 84)
        self.linear2 = nn.Linear(84, 10)
        self.to(device)
        
    def forward(self, x): 
        x = self.relu(self.conv1(x)) 
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x)) 
        x = x.reshape(x.shape[0], -1)
        x = self.relu(self.linear1(x))
        x = self.linear2(x)
        return x
    
num_epochs = 2 
learning_rate = 1e-3
    
model = LeNet()
summary(model, (1, 28, 28))
train(model, train_loader, num_epochs, learning_rate)
print(f'test: {accuracy(model, test_loader):.1%}')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
              ReLU-2            [-1, 6, 28, 28]               0
         AvgPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 16, 10, 10]           2,416
              ReLU-5           [-1, 16, 10, 10]               0
         AvgPool2d-6             [-1, 16, 5, 5]               0
            Conv2d-7            [-1, 120, 1, 1]          48,120
              ReLU-8            [-1, 120, 1, 1]               0
            Linear-9                   [-1, 84]          10,164
             ReLU-10                   [-1, 84]               0
           Linear-11                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

In [129]:
class VGG(nn.Module):
    def __init__(self, architecture, in_channels=3, in_size=(224, 224)):
        super(VGG, self).__init__()
        self.cnv = self.create_conv_layers(in_channels, architecture[0])
        out_size = torch.tensor(in_size) // 2 ** architecture[0].count('M')
        self.fcs = self.create_fcs_layers(architecture[0][-2] * out_size.prod(), architecture[1])
        self.to(device)
        
    def forward(self, x):
        x = self.cnv(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fcs(x)
        return x
        
    def create_conv_layers(self, in_channels, architecture):
        layers = []
        for out in architecture:
            if type(out) == int:
                layers += [
                    nn.Conv2d(in_channels, out, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                    nn.BatchNorm2d(out),
                    nn.ReLU()
                ]
                in_channels = out
            elif out == 'M':
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]
        return nn.Sequential(*layers)
    
    def create_fcs_layers(self, in_channels, architecture):
        layers = []
        for out in architecture[:-1]:
            layers += [
                nn.Linear(in_channels, out),
                nn.ReLU(),
                nn.Dropout(p=0.5)
            ]
            in_channels = out
        layers += [nn.Linear(in_channels, architecture[-1])]
        return nn.Sequential(*layers)
        
VGG16 = [
    [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    [4096, 4096, 1000]
]
        
model = VGG(VGG16)
x = torch.randn(1, 3, 224, 224).to(device)
# print(model)
# print(model(x).shape)
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,928
       BatchNorm2d-5         [-1, 64, 224, 224]             128
              ReLU-6         [-1, 64, 224, 224]               0
         MaxPool2d-7         [-1, 64, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]          73,856
       BatchNorm2d-9        [-1, 128, 112, 112]             256
             ReLU-10        [-1, 128, 112, 112]               0
           Conv2d-11        [-1, 128, 112, 112]         147,584
      BatchNorm2d-12        [-1, 128, 112, 112]             256
             ReLU-13        [-1, 128, 112, 112]               0
        MaxPool2d-14          [-1, 128,

In [136]:
VGG16 = [
    [8, 8, 'M', 16, 16, 'M'],
    [64, 10]
]

model = VGG(VGG16, in_channels=1, in_size=(28, 28))
summary(model, (1, 28, 28))
train(model, train_loader, num_epochs, learning_rate)
print(f'test: {accuracy(model, test_loader):.1%}')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
              ReLU-3            [-1, 8, 28, 28]               0
            Conv2d-4            [-1, 8, 28, 28]             584
       BatchNorm2d-5            [-1, 8, 28, 28]              16
              ReLU-6            [-1, 8, 28, 28]               0
         MaxPool2d-7            [-1, 8, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           1,168
       BatchNorm2d-9           [-1, 16, 14, 14]              32
             ReLU-10           [-1, 16, 14, 14]               0
           Conv2d-11           [-1, 16, 14, 14]           2,320
      BatchNorm2d-12           [-1, 16, 14, 14]              32
             ReLU-13           [-1, 16, 14, 14]               0
        MaxPool2d-14             [-1, 1